SELF NORMALIZING NEURAL NETWORKS : GROUP ASSIGNMENT
    
                                   SAURAV RAI
                                    
                                   17558

IMPORTING IMPORTANT LIBRARIES :

In [1]:
import numpy as np
import pandas as pd

In [2]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

In [3]:
import os
import time

In [4]:
from __future__ import absolute_import , division ,print_function
import numbers

In [5]:
from tensorflow.contrib import layers
from tensorflow.python.framework import ops
from tensorflow.python.framework import tensor_shape
from tensorflow.python.framework import tensor_util
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import random_ops
from tensorflow.python.ops import array_ops
from tensorflow.python.layers import utils

/usr/local/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [6]:
from subprocess import check_output
print(check_output(["ls","/home/sauravrai/Desktop/ACADEMICS/semesterII/machine_learning/mac_lab/group_assignment/fashion_mnist"]).decode("utf8"))

display_netwok.py.ipynb
fashion-mnist_test.csv
fashion-mnist_train.csv
gradient.py.ipynb
load_MNIST.py.ipynb
neuralNetwork.py.ipynb
sample_images.py.ipynb
train.py.ipynb



TRAINING AND TESTING THE IMAGES

In [7]:
df_train = pd.read_csv("/home/sauravrai/Desktop/ACADEMICS/semesterII/machine_learning/mac_lab/group_assignment/fashion_mnist/fashion-mnist_train.csv")

In [8]:
df_test = pd.read_csv("/home/sauravrai/Desktop/ACADEMICS/semesterII/machine_learning/mac_lab/group_assignment/fashion_mnist/fashion-mnist_test.csv")

In [9]:
train_x = df_train[df_train.columns[1:]]
# 60000 rows × 784 columns

In [10]:
train_y = df_train["label"] 
# SIZE OF ABOUT (60000,)


In [11]:
test_x = df_test[df_test.columns[1:]] #size is (10000,784)
test_y = df_test["label"] # size is (10000,)

train_y.shape[0] is 60000

np.arrange will Return evenly spaced values within a given interval

index_offset = np.arange(60000) * 10

array([     0,     10,     20, ..., 599970, 599980, 599990])

np.ravel : Return a contiguous flattened array(means it will make a continous flat array)



In [12]:
def dense_to_one_hot(labels_dense , num_classes=10):
    num_labels = labels_dense.shape[0] # 60000 which is nothing but labels
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels ,num_classes)) #array of size (60000,784)
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    return pd.DataFrame(labels_one_hot)

In [13]:
train_y = dense_to_one_hot(train_y, num_classes=10)
test_y = dense_to_one_hot(test_y, num_classes=10)

HYPER-PARAMETERS

In [14]:
#parameters
learning_rate = 0.05
training_epochs = 10
batch_size = 100
display_step = 1

#Networks parameters
n_hidden_1 = 784 #This is the 1st layer number of features
n_hidden_2 = 784 #This is the 2nd layer number of features 
n_input = 784 #FMNIST data input (image shape 28 * 28)
n_classes = 10 #FMNIST total classes (0-9 digits)




In [15]:
#tf Graph input
x = tf.placeholder("float",[None ,n_input])
y = tf.placeholder("float",[None ,n_classes])
dropoutRate = tf.placeholder(tf.float32) # The type of elements in the tensor to be fed.
is_training= tf.placeholder(tf.bool)

In [16]:
def selu(x):
    with ops.name_scope('selu') as scope:
        alpha = 1.6732
        scale = 1.0507
        return scale * tf.where ( x>=0 , x , alpha * tf.nn.elu(x) )
    

In [17]:
def dropout_selu(x, rate, alpha= -1.7580993408473766, fixedPointMean=0.0, fixedPointVar=1.0, 
                 noise_shape=None, seed=None, name=None, training=False):
    """Dropout to a value with rescaling."""

    def dropout_selu_impl(x, rate, alpha, noise_shape, seed, name):
        keep_prob = 1.0 - rate
        x = ops.convert_to_tensor(x, name="x")
        if isinstance(keep_prob, numbers.Real) and not 0 < keep_prob <= 1:
            raise ValueError("keep_prob must be a scalar tensor or a float in the "
                                             "range (0, 1], got %g" % keep_prob)
        keep_prob = ops.convert_to_tensor(keep_prob, dtype=x.dtype, name="keep_prob")
        keep_prob.get_shape().assert_is_compatible_with(tensor_shape.scalar())

        alpha = ops.convert_to_tensor(alpha, dtype=x.dtype, name="alpha")
        keep_prob.get_shape().assert_is_compatible_with(tensor_shape.scalar())

        if tensor_util.constant_value(keep_prob) == 1:
            return x

        noise_shape = noise_shape if noise_shape is not None else array_ops.shape(x)
        random_tensor = keep_prob
        random_tensor += random_ops.random_uniform(noise_shape, seed=seed, dtype=x.dtype)
        binary_tensor = math_ops.floor(random_tensor)
        ret = x * binary_tensor + alpha * (1-binary_tensor)

        a = tf.sqrt(fixedPointVar / (keep_prob *((1-keep_prob) * tf.pow(alpha-fixedPointMean,2) + fixedPointVar)))

        b = fixedPointMean - a * (keep_prob * fixedPointMean + (1 - keep_prob) * alpha)
        ret = a * ret + b
        ret.set_shape(x.get_shape())
        return ret

    with ops.name_scope(name, "dropout", [x]) as name:
        return utils.smart_cond(training,
            lambda: dropout_selu_impl(x, rate, alpha, noise_shape, seed, name),
            lambda: array_ops.identity(x))

In [18]:
# (1) Scale input to zero mean and unit variance
scaler = StandardScaler().fit(train_x)
test_x = scaler.transform(test_x)


In [19]:
# Create model
def multilayer_perceptron(x, weights, biases, rate, is_training):
    # Hidden layer with SELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = selu(layer_1)
    layer_1 = dropout_selu(layer_1,rate, training=is_training)
    
    # Hidden layer with SELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = selu(layer_2)
    layer_2 = dropout_selu(layer_2,rate, training=is_training)

    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [20]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1],stddev=np.sqrt(1/n_input))),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2],stddev=np.sqrt(1/n_hidden_1))),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes],stddev=np.sqrt(1/n_hidden_2)))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1],stddev=0)),
    'b2': tf.Variable(tf.random_normal([n_hidden_2],stddev=0)),
    'out': tf.Variable(tf.random_normal([n_classes],stddev=0))
}
# Construct model
pred = multilayer_perceptron(x, weights, biases, rate=dropoutRate, is_training=is_training)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate).minimize(cost)

 # Test model
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
# Calculate accuracy
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
         
# Initializing the variables
init = tf.global_variables_initializer()

In [21]:
# Launch the graph
gpu_options = tf.GPUOptions(allow_growth=True)

with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
    sess.run(init)

    start_time = time.time()
    
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(train_x.shape[0]/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            randidx = np.random.randint(len(train_x), size=batch_size)
            batch_xs = train_x.iloc[randidx]
            batch_ys = train_y.iloc[randidx]
            batch_xs = scaler.transform(batch_xs)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys, dropoutRate: 0.05, is_training:True})

            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print ("Epoch:", '%04d' % (epoch+1), "cost=","{:.9f}".format(avg_cost))
            
            accTrain, costTrain = sess.run([accuracy, cost], 
                                                        feed_dict={x: batch_xs, y: batch_ys, 
                                                                   dropoutRate: 0.0, is_training:False})
            
            print("Train-Accuracy:", accTrain,"Train-Loss:", costTrain)

            accTest, costVal = sess.run([accuracy, cost], 
                                                     feed_dict={x: test_x, y: test_y, 
                                                                   dropoutRate: 0.0, is_training:False})

            print("Test-Accuracy:", accTest,"\n")
            
    print("--- %s seconds ---" % (time.time() - start_time))

Epoch: 0001 cost= 0.540397569
Train-Accuracy: 0.9 Train-Loss: 0.24052894
Test-Accuracy: 0.8454 

Epoch: 0002 cost= 0.408464546
Train-Accuracy: 0.96 Train-Loss: 0.13915735
Test-Accuracy: 0.8564 

Epoch: 0003 cost= 0.350047752
Train-Accuracy: 0.98 Train-Loss: 0.13835353
Test-Accuracy: 0.8726 

Epoch: 0004 cost= 0.328777448
Train-Accuracy: 0.96 Train-Loss: 0.08928568
Test-Accuracy: 0.8729 

Epoch: 0005 cost= 0.301540912
Train-Accuracy: 0.96 Train-Loss: 0.11743866
Test-Accuracy: 0.8801 

Epoch: 0006 cost= 0.282150423
Train-Accuracy: 0.97 Train-Loss: 0.15421754
Test-Accuracy: 0.8831 

Epoch: 0007 cost= 0.264087401
Train-Accuracy: 0.97 Train-Loss: 0.06833842
Test-Accuracy: 0.8883 

Epoch: 0008 cost= 0.247741675
Train-Accuracy: 0.99 Train-Loss: 0.084286645
Test-Accuracy: 0.8915 

Epoch: 0009 cost= 0.236605111
Train-Accuracy: 0.98 Train-Loss: 0.073307745
Test-Accuracy: 0.8909 

Epoch: 0010 cost= 0.226757697
Train-Accuracy: 0.98 Train-Loss: 0.06030463
Test-Accuracy: 0.8921 

--- 118.22559189796